# Piqer Perplexity Playbook

<p>
Mal Minhas, v0.6<br>
11.11.24
</p>
<p>
<h4>Versions</h4>
<ul>
<li><b>v0.1</b>: 11.11.24. First version</li>
</ul>
</p>

### 1. Introduction

This notebook builds off a conversation with Tim earlier today and his work on [piqer](https://www.piqer.com).  What is outlined here is a quick recipe whereby we pull down the articles in a feed using piqer and then generate a summary of each one.

### 2. Retrieving the feed article links using piqer

In [1]:
from IPython.display import display, Markdown
from bs4 import BeautifulSoup
import requests
import os

def getPiqerArticleUrls(target):
    url = f"https://www.piqer.com/rss/{target}"
    r = requests.get(url)
    assert(r.status_code == 200)
    html_content = r.text
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')
    # Find all <a> tags within the list and extract href attributes
    article_urls = [a['href'] for a in soup.find_all('a', href=True)]    
    return article_urls

feed = "climatecto.substack.com/feed"
articles = getPiqerArticleUrls(feed)
print(f"Piqer found {len(articles)} articles at {feed}")
print(f"First article is at '{articles[0]}'")

Piqer found 1 articles at climatecto.substack.com/feed
First article is at 'https://climatecto.substack.com/p/the-true-nature-of-the-crisis'


### 3. Generating a summary of each article along with citations

In [2]:
API_KEY = os.environ.get('PERPLEXITY_API_KEY')
DEFAULT_MODEL = 'llama-3.1-sonar-small-128k-online'
DEFAULT_TOKENS = 1024
CONCISE_DETAIL = 'Be precise and concise.'
MID_LEVEL_DETAIL = 'Explain in a few sentences.'
GREAT_DETAIL = 'Explain in a lot of detail.'
POLITE_DETAIL = 'You are an artificial intelligence assistant and you need to engage in a helpful, detailed, polite conversation with a user.'
DEFAULT_TEMP = 0.0

def renderTextAsMarkdown(text: str):
    display(Markdown(text))

def getPerplexityResponse(question:str, model:str=DEFAULT_MODEL, direction:str=MID_LEVEL_DETAIL, max_tokens:int=DEFAULT_TOKENS, temperature:float=DEFAULT_TEMP) -> str:
    url = "https://api.perplexity.ai/chat/completions"
    payload = {
        "model": model,
        "stream": False,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "messages": [
            {
                "role": "system",
                "content": f"{direction}"
            },
            {
                "role": "user",
                "content": question
            }
        ]
    }
    headers = {
        "accept": "application/json",
        "content-type": "application/json",
        "Authorization": f"Bearer {API_KEY}",
    }
    r = requests.post(url, json=payload, headers=headers)
    assert(r.status_code == 200)
    return r.json()

def renderResponse(d):
    return renderTextAsMarkdown(d.get("choices")[0].get("message").get("content"))

def getCitations(d):
    def formatReferences(url_list):
        return [f"[{i + 1}]: {url}" for i, url in enumerate(url_list)]
    for ref in formatReferences(d.get("citations")):
        display(ref)

In [3]:
def generateSummary(url):
    prompt = f"Please generate a summary of the article at this URL which captures its key themes and takeaways: {url}"
    d = getPerplexityResponse(prompt)
    renderResponse(d)
    return d

Now we can invoke `generateSummary` to generate a summary of the required article and render it as Markdown:

In [4]:
%%time

d = generateSummary(articles[0])

The article "The True Nature of the Crisis" on the Climate CTO Substack discusses the complexities and misconceptions surrounding the climate crisis. Here are the key themes and takeaways:

1. **Misconceptions About Climate Change**:
   - The article challenges common beliefs about global warming and climate change, suggesting that much of what is known is incorrect or oversimplified[1].

2. **Renewable Energy and Collective Action**:
   - It emphasizes the importance of shifting towards renewable energy and collective action to combat climate change, highlighting that this fight is ongoing and demands sustained effort[2].

3. **The Real Cost of Climate Failure**:
   - The article underscores the severe consequences of failing to address climate change, including unprecedented societal changes and increased risk of mass displacement events[3].

4. **Economic Systems and Environmental Impact**:
   - It critiques neoliberal economics for prioritizing growth over sustainability, arguing that this approach exacerbates the environmental crisis by perpetuating excessive energy use and material transformation[5].

5. **Authoritarianism and Climate Crisis**:
   - The article warns about the potential for authoritarian demagogues to exploit the climate crisis for political gain, using scapegoats like refugees to activate nationalist sentiments[4].

6. **Systemic Change and Resilience**:
   - It advocates for systemic change and fostering local resilience to mitigate the effects of the climate crisis, emphasizing the need for a sustainable future that prioritizes human wellbeing over economic growth[2][5].

CPU times: user 11.2 ms, sys: 2.78 ms, total: 13.9 ms
Wall time: 3.48 s


Let's pull out the citations:

In [5]:
getCitations(d)

'[1]: https://richardcrim.substack.com'

'[2]: https://rickylanusse.substack.com/p/the-one-reason-why-the-climate-crisis-3a6/comments'

'[3]: https://alexsteffen.substack.com/p/the-real-cost-of-climate-failure'

'[4]: https://davekarpf.substack.com/p/will-the-climate-crisis-be-a-boon'

'[5]: https://www.planetcritical.com/p/the-world-is-in-crisis'

These look wonky and incomplete.